In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/GAMENet/code
!ls

/content/drive/.shortcut-targets-by-id/1kkPpZE8ZtC_X78y5dCl2uzXTlWLbTrsp/GAMENet/code
baseline    layers.py  __pycache__	    train_GAMENet.py
deplicated  models.py  train_GAMENet.ipynb  util.py


In [3]:
!pip install dill dnc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached dnc-1.1.0-py3-none-any.whl (20 kB)
  Using cached flann-1.6.13-py3-none-any.whl (24 kB)


In [4]:
import torch
import argparse
import numpy as np
import dill
import time
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import os
import torch.nn.functional as F
from collections import defaultdict

from models import GAMENet
from util import llprint, multi_label_metric, ddi_rate_score, get_n_params

torch.manual_seed(1203)
np.random.seed(1203)

model_name = 'GAMENet'
resume_name = ''
ddi = True #train with ddi graph

In [5]:
if not os.path.exists(os.path.join("saved", model_name)):
  os.makedirs(os.path.join("saved", model_name))

data_path = '../data/records_final.pkl'
voc_path = '../data/voc_final.pkl'

ehr_adj_path = '../data/ehr_adj_final.pkl'
ddi_adj_path = '../data/ddi_A_final.pkl'
device = torch.device('cuda:0')

ehr_adj = dill.load(open(ehr_adj_path, 'rb'))
ddi_adj = dill.load(open(ddi_adj_path, 'rb'))
data = dill.load(open(data_path, 'rb'))
voc = dill.load(open(voc_path, 'rb'))
diag_voc, pro_voc, med_voc = voc['diag_voc'], voc['pro_voc'], voc['med_voc']

split_point = int(len(data) * 2 / 3)
data_train = data[:split_point]
eval_len = int(len(data[split_point:]) / 2)
data_test = data[split_point:split_point + eval_len]
data_eval = data[split_point+eval_len:]
voc_size = (len(diag_voc.idx2word), len(pro_voc.idx2word), len(med_voc.idx2word))
model = GAMENet(voc_size, ehr_adj, ddi_adj, emb_dim=64, device=device, ddi_in_memory=ddi)
model.to(device=device)

print('parameters', get_n_params(model))
LR = 0.0002
optimizer = Adam(list(model.parameters()), lr=LR)

parameters 452434


In [6]:
def eval(model, data_eval, voc_size, epoch):
    # evaluate
    print('')
    model.eval()
    smm_record = []
    ja, prauc, avg_p, avg_r, avg_f1 = [[] for _ in range(5)]
    case_study = defaultdict(dict)
    med_cnt = 0
    visit_cnt = 0
    for step, input in enumerate(data_eval):
        y_gt = []
        y_pred = []
        y_pred_prob = []
        y_pred_label = []
        for adm_idx, adm in enumerate(input):

            target_output1 = model(input[:adm_idx+1])

            y_gt_tmp = np.zeros(voc_size[2])
            y_gt_tmp[adm[2]] = 1
            y_gt.append(y_gt_tmp)

            target_output1 = F.sigmoid(target_output1).detach().cpu().numpy()[0]
            y_pred_prob.append(target_output1)
            y_pred_tmp = target_output1.copy()
            y_pred_tmp[y_pred_tmp>=0.5] = 1
            y_pred_tmp[y_pred_tmp<0.5] = 0
            y_pred.append(y_pred_tmp)
            y_pred_label_tmp = np.where(y_pred_tmp == 1)[0]
            y_pred_label.append(sorted(y_pred_label_tmp))
            visit_cnt += 1
            med_cnt += len(y_pred_label_tmp)


        smm_record.append(y_pred_label)
        adm_ja, adm_prauc, adm_avg_p, adm_avg_r, adm_avg_f1 = multi_label_metric(np.array(y_gt), np.array(y_pred), np.array(y_pred_prob))
        case_study[adm_ja] = {'ja': adm_ja, 'patient': input, 'y_label': y_pred_label}

        ja.append(adm_ja)
        prauc.append(adm_prauc)
        avg_p.append(adm_avg_p)
        avg_r.append(adm_avg_r)
        avg_f1.append(adm_avg_f1)
        llprint('\rEval--Epoch: %d, Step: %d/%d' % (epoch, step, len(data_eval)))

    # ddi rate
    ddi_rate = ddi_rate_score(smm_record)

    llprint('\tDDI Rate: %.4f, Jaccard: %.4f,  PRAUC: %.4f, AVG_PRC: %.4f, AVG_RECALL: %.4f, AVG_F1: %.4f\n' % (
        ddi_rate, np.mean(ja), np.mean(prauc), np.mean(avg_p), np.mean(avg_r), np.mean(avg_f1)
    ))
    dill.dump(obj=smm_record, file=open('../data/gamenet_records.pkl', 'wb'))
    dill.dump(case_study, open(os.path.join('saved', model_name, 'case_study.pkl'), 'wb'))

    # print('avg med', med_cnt / visit_cnt)

    return ddi_rate, np.mean(ja), np.mean(prauc), np.mean(avg_p), np.mean(avg_r), np.mean(avg_f1)

#eval(model, data_test, voc_size, 0)

In [7]:
TEST = False
if TEST:
  model.load_state_dict(torch.load(open(resume_name, 'rb')))
  model.to(device=device)
  eval(model, data_test, voc_size, 0)

In [8]:
def train(model, data_train, ddi=True):
  EPOCH = 40

  Neg_Loss = ddi
  DDI_IN_MEM = ddi
  TARGET_DDI = 0.05
  T = 0.5
  decay_weight = 0.85

  history = defaultdict(list)
  best_epoch = 0
  best_ja = 0
  start_time = time.time()
  for epoch in range(EPOCH):
    loss_record1 = []
    epoch_time = time.time()
    model.train()
    prediction_loss_cnt = 0
    neg_loss_cnt = 0
    for step, input in enumerate(data_train):
      for idx, adm in enumerate(input):
        seq_input = input[:idx+1]
        loss1_target = np.zeros((1, voc_size[2]))
        loss1_target[:, adm[2]] = 1
        loss3_target = np.full((1, voc_size[2]), -1)
        for idx, item in enumerate(adm[2]):
          loss3_target[0][idx] = item

        target_output1, batch_neg_loss = model(seq_input)

        loss1 = F.binary_cross_entropy_with_logits(target_output1, torch.FloatTensor(loss1_target).to(device))
        loss3 = F.multilabel_margin_loss(F.sigmoid(target_output1), torch.LongTensor(loss3_target).to(device))
        if Neg_Loss:
          target_output1 = F.sigmoid(target_output1).detach().cpu().numpy()[0]
          target_output1[target_output1 >= 0.5] = 1
          target_output1[target_output1 < 0.5] = 0
          y_label = np.where(target_output1 == 1)[0]
          current_ddi_rate = ddi_rate_score([[y_label]])
          if current_ddi_rate <= TARGET_DDI:
            loss = 0.9 * loss1 + 0.01 * loss3
            prediction_loss_cnt += 1
          else:
            rnd = np.exp((TARGET_DDI - current_ddi_rate)/T)
            if np.random.rand(1) < rnd:
              loss = batch_neg_loss
              neg_loss_cnt += 1
            else:
              loss = 0.9 * loss1 + 0.01 * loss3
              prediction_loss_cnt += 1
        else:
          loss = 0.9 * loss1 + 0.01 * loss3

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        loss_record1.append(loss.item())

      llprint('\rTrain--Epoch: %d, Step: %d/%d, L_p cnt: %d, L_neg cnt: %d' % (epoch, step, len(data_train), prediction_loss_cnt, neg_loss_cnt))
    # annealing
    T *= decay_weight

    ddi_rate, ja, prauc, avg_p, avg_r, avg_f1 = eval(model, data_eval, voc_size, epoch)

    history['ja'].append(ja)
    history['ddi_rate'].append(ddi_rate)
    history['avg_p'].append(avg_p)
    history['avg_r'].append(avg_r)
    history['avg_f1'].append(avg_f1)
    history['prauc'].append(prauc)

    end_time = time.time()
    elapsed_time = (end_time - epoch_time) / 60
    llprint('\tEpoch: %d, Loss: %.4f, One Epoch Time: %.2fm, Appro Left Time: %.2fh\n' % (epoch,
                                                                                          np.mean(loss_record1),
                                                                                          elapsed_time,
                                                                                          elapsed_time * (
                                                                                            EPOCH - epoch - 1)/60))

    torch.save(model.state_dict(), open( os.path.join('saved', model_name, 'Epoch_%d_JA_%.4f_DDI_%.4f.model' % (epoch, ja, ddi_rate)), 'wb'))
    print('')
    if epoch != 0 and best_ja < ja:
      best_epoch = epoch
      best_ja = ja


  dill.dump(history, open(os.path.join('saved', model_name, 'history.pkl'), 'wb'))

  # test
  torch.save(model.state_dict(), open(
    os.path.join('saved', model_name, 'final.model'), 'wb'))

  total_time = (time.time() - start_time) / 60 / 60 
  print('best_epoch:', best_epoch)
  print(f'total time: {total_time}h')

train(model, data_train, ddi)

Train--Epoch: 0, Step: 4232/4233, L_p cnt: 4696, L_neg cnt: 5779
Eval--Epoch: 0, Step: 1058/1059	DDI Rate: 0.0889, Jaccard: 0.3534,  PRAUC: 0.6211, AVG_PRC: 0.6356, AVG_RECALL: 0.4584, AVG_F1: 0.5141
	Epoch: 0, Loss: 0.1166, One Epoch Time: 2.50m, Appro Left Time: 1.62h

Train--Epoch: 1, Step: 4232/4233, L_p cnt: 5540, L_neg cnt: 4935
Eval--Epoch: 1, Step: 1058/1059	DDI Rate: 0.0569, Jaccard: 0.3679,  PRAUC: 0.6292, AVG_PRC: 0.6368, AVG_RECALL: 0.4824, AVG_F1: 0.5290
	Epoch: 1, Loss: 0.1329, One Epoch Time: 2.23m, Appro Left Time: 1.41h

Train--Epoch: 2, Step: 4232/4233, L_p cnt: 6830, L_neg cnt: 3645
Eval--Epoch: 2, Step: 1058/1059	DDI Rate: 0.0546, Jaccard: 0.3756,  PRAUC: 0.6373, AVG_PRC: 0.6455, AVG_RECALL: 0.4854, AVG_F1: 0.5369
	Epoch: 2, Loss: 0.1587, One Epoch Time: 2.34m, Appro Left Time: 1.44h

Train--Epoch: 3, Step: 4232/4233, L_p cnt: 6731, L_neg cnt: 3744
Eval--Epoch: 3, Step: 1058/1059	DDI Rate: 0.0541, Jaccard: 0.3927,  PRAUC: 0.6510, AVG_PRC: 0.6574, AVG_RECALL: 0.5011,